In [2]:
import pandas as pd
from mtranslate import translate
from tqdm.notebook import tqdm

In [3]:
df=pd.read_csv("./dataSet/Sentiment_Analysis.csv")

In [23]:
df.shape

(40000, 5)

In [15]:
splitted=df.content[1].split()
filtered=' '.join([word for word in splitted if not word.startswith('@')])
print(filtered)

  

Layin n bed with a headache ughhhh...waitin on your call...


In [16]:
df['tweets']=df.content.apply(lambda x:' '.join([word for word in x.split() if not word.startswith('@')]))

In [19]:
df.to_csv("./dataSet/tweets.csv",index=False)

### getting unrepeated sentiments

In [26]:
sent={}
for sentiment in df.sentiment.unique():
    sent[sentiment]=translate(sentiment,'ckb')

In [27]:
sent

{'empty': 'بەتاڵ',
 'sadness': 'دڵتەنگی',
 'enthusiasm': 'پەرۆشی',
 'neutral': 'بێلایەن',
 'worry': 'نیگەرانی',
 'surprise': 'سوپرایس',
 'love': 'خۆشەویستی',
 'fun': 'خۆش',
 'hate': 'ڕق',
 'happiness': 'بەختەوەری',
 'boredom': 'بێزاری',
 'relief': 'حەسانەوە',
 'anger': 'تووڕەیی'}

In [39]:
sentiments={'empty': 'بەتاڵ',
 'sadness': 'دڵتەنگ',
 'enthusiasm': 'پەرۆش',
 'neutral': 'ئاسایی',
 'worry': 'نیگەران',
 'surprise': 'سوپرایز',
 'love': 'خۆشەویستی',
 'fun': 'خۆشی',
 'hate': 'ڕق',
 'happiness': 'بەختەوەر',
 'boredom': 'بێزار',
 'relief': 'ئارام',
 'anger': 'تووڕە'}

In [40]:
df['kurdish_sentiments']=df.sentiment.apply(lambda x:sentiments[x])
sentiments["empty"]

'بەتاڵ'

In [41]:
df['kurdish_sentiments']

0            بەتاڵ
1           دڵتەنگ
2           دڵتەنگ
3            پەرۆش
4           ئاسایی
           ...    
39995       ئاسایی
39996    خۆشەویستی
39997    خۆشەویستی
39998     بەختەوەر
39999    خۆشەویستی
Name: kurdish_sentiments, Length: 40000, dtype: object

In [44]:
df.head()

,tweet_id,sentiment,author,content,tweets,kurdish_sentiments
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...,بەتاڵ
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin on...,دڵتەنگ
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,دڵتەنگ
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,پەرۆش
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...,ئاسایی


In [2]:
df.to_csv("./dataSet/Tweets_Sentiment.csv",index=False)

NameError: name 'df' is not defined

### Translating tweets

In [2]:
import pandas as pd
from mtranslate import translate
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
partial_save_amount = 1000

def translate_text(text, to_language='ckb'):
    try:
        return translate(text, to_language)
    except Exception as e:
        print(f'Error translating text: {text} with error {e}')
        return None

def dataset_translator(df):
    res = {}
    count = 0
    futures = []
    with ThreadPoolExecutor(max_workers=10) as executor: 
        for i, row in df.iterrows():
            futures.append(executor.submit(translate_text, row['tweets'], 'ckb'))

        for future in tqdm(as_completed(futures), total=len(futures)):
            count += 1
            result = future.result()
            if result is not None:
                myid = df.iloc[count - 1]['tweet_id']  # Adjust index to match future result
                res[myid] = result

            if count % partial_save_amount == 0:
                results = pd.DataFrame.from_dict(res, orient='index')
                results = results.reset_index().rename(columns={'index': 'tweet_id', 0: 'kurdish_tweets'})
                results = results.merge(df.iloc[:len(res)], on='tweet_id', how='right')
                results.to_csv(f"./dataSet/translated/kurdish_tweets-{len(res)}.csv", index=False)
                print(f'saved at {len(res)}')
                time.sleep(30)

    # After all futures are completed, save any remaining translations
    if res:
        results = pd.DataFrame.from_dict(res, orient='index')
        results = results.reset_index().rename(columns={'index': 'tweet_id', 0: 'kurdish_tweets'})
        results = results.merge(df, on='tweet_id', how='right')
        results.to_csv(f"./dataSet/translated/kurdish_tweets_final.csv", index=False)
        print(f'Final save at {len(res)}')

    return pd.DataFrame.from_dict(res, orient='index')

In [3]:
data=pd.read_csv("./dataSet/Tweets_Sentiment.csv")


In [5]:
kurdish_dataset = dataset_translator(data.iloc[1998:])


  0%|          | 0/40000 [00:00<?, ?it/s]

Error translating text: nan with error quote_from_bytes() expected bytes
Error translating text: nan with error quote_from_bytes() expected bytes
saved at 998
saved at 1998
Error translating text: E docs are getting younger these days.can u guess where am i??yup, NUH!! paying my ear doc a visit. Ouch!! with error HTTP Error 429: Too Many Requests
Error translating text: aw were sad you had to leave tokyo. come back.please with error HTTP Error 429: Too Many Requests
Error translating text: Lucky Rob... always gettin' the T8's from Emalon with error HTTP Error 429: Too Many Requests
Error translating text: prepare yourself with error HTTP Error 429: Too Many Requests
Error translating text: haha! nice! Have fun doll! Hopefully I'll see ya soon! I miss ur face! with error HTTP Error 429: Too Many Requests
Error translating text: there are # days of summervac &amp; school comes along just 2 end it &amp; the annual problem of r generation is finding a good way 2 spend it!! with error HTTP 

,tweet_id,sentiment,author,content,tweets,kurdish_sentiments
1998,1957449197,sadness,Keiralou,ouch epic bruise between my toes and a lump j...,ouch epic bruise between my toes and a lump ju...,دڵتەنگ
1999,1957450153,neutral,vniarbcca,got GREATscores. i'm off a while for exam yeaps.,got GREATscores. i'm off a while for exam yeaps.,ئاسایی
2000,1957450274,sadness,akulbanskiy,Epic ocd moment? I deleted my sims file that i...,Epic ocd moment? I deleted my sims file that i...,دڵتەنگ
2001,1957451066,worry,LewisBeechey,@MrDix ugly. What programmes do you have open?,ugly. What programmes do you have open?,نیگەران
2002,1957451307,surprise,mimidang,Owww I'm aching alll over!!,Owww I'm aching alll over!!,سوپرایز
...,...,...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor,NaN,ئاسایی
39996,1753919001,love,drapeaux,Happy Mothers Day All my love,Happy Mothers Day All my love,خۆشەویستی
39997,1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...,Happy Mother's Day to all the mommies out ther...,خۆشەویستی
39998,1753919043,happiness,ipdaman1,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY NE...,بەختەوەر
